In [ ]:
!git clone https://github.com/VAST-AI-Research/TripoSR.git
%cd TripoSR

fatal: destination path 'TripoSR' already exists and is not an empty directory.
/content/TripoSR


In [ ]:
!pip install -r requirements.txt

  Cloning https://github.com/tatsy/torchmcubes.git to /tmp/pip-req-build-0c4kf0zu
  Running command git clone --filter=blob:none --quiet https://github.com/tatsy/torchmcubes.git /tmp/pip-req-build-0c4kf0zu
  Resolved https://github.com/tatsy/torchmcubes.git to commit 3381600ddc3d2e4d74222f8495866be5fafbace4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached transformers-4.35.0-py3-none-any.whl.metadata (123 kB)
  Using cached tokenizers-0.14.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.7 kB)
  Using cached huggingface_hub-0.17.3-py3-none-any.whl.metadata (13 kB)
Using cached transformers-4.35.0-py3-none-any.whl (7.9 MB)
Using cached tokenizers-0.14.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.8 MB)
Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface

In [ ]:
!pip install onnxruntime
!pip install rembg torch xatlas pillow numpy

In [ ]:
!pip install --upgrade huggingface_hub transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.1/512.1 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 128.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 110.6 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.3
    Uninstalling huggingface-hub-0.17.3:
      Successfully uninstalled huggingface-hub-0.17.3
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.14.1
    Uninstalling tokenizers-0.14.1:
      Successfully uninstalled tokenizers-0.14.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.0
    Uninstalling transformers-4.35.0:
      Successfully uninstalled transformers-4.35.0


In [ ]:
# STEP 1: Install required libraries
!pip install pyngrok flask-ngrok

# STEP 2: Set up ngrok auth token
from pyngrok import ngrok
ngrok.set_auth_token("2tIu4eZ8fEFeUyjchaRTT3qlQx2_6WwpBMEybrtqmDzqUNwMM")  # Replace with your token

# STEP 3: Imports and App Setup
from flask import Flask, request, jsonify, send_from_directory
import os
import subprocess
import threading
import uuid
import time

app = Flask(__name__)
UPLOAD_FOLDER = '/content/uploads'
OUTPUT_FOLDER = '/content/output'
os.makedirs(UPLOAD_FOLDER, exist_ok=True)
os.makedirs(OUTPUT_FOLDER, exist_ok=True)

# STEP 4: Ngrok Public URL (extract the public URL string)
tunnel = ngrok.connect(5000)
public_url = tunnel.public_url
print(f"\n🚀 Public API endpoint: {public_url}/convert")

# STEP 5: Dummy root endpoint
@app.route('/')
def root():
    return jsonify({"message": "Server is running. Use /convert to post your image."})

# STEP 6: Conversion Logic
def convert_image_to_3d_glb(input_image_path, job_id):
    output_dir = f"{OUTPUT_FOLDER}/{job_id}"
    os.makedirs(output_dir, exist_ok=True)
    command = [
        'python', '/content/TripoSR/run.py',  # Make sure this file exists and works
        input_image_path,
        '--device', 'cuda:0',
        '--output-dir', output_dir,
        '--model-save-format', 'glb'
    ]
    subprocess.run(command, check=True)
    return f"{output_dir}/0/mesh.glb"  # Assuming this structure is correct

# STEP 7: Upload and Convert Endpoint
@app.route('/convert', methods=['POST'])
def convert():
    if 'image' not in request.files:
        return jsonify({'error': 'No image file found'}), 400

    image = request.files['image']
    job_id = str(uuid.uuid4())[:8]
    input_path = os.path.join(UPLOAD_FOLDER, f"{job_id}.jpg")
    image.save(input_path)
    print(f"📸 Image saved: {input_path}, Job ID: {job_id}")

    try:
        glb_path = convert_image_to_3d_glb(input_path, job_id)
    except subprocess.CalledProcessError as e:
        print(f"❌ Conversion failed: {e}")
        return jsonify({'error': '3D conversion failed', 'details': str(e)}), 500

    glb_filename = os.path.basename(glb_path)
    glb_url = f"{public_url}/download/{job_id}/0/{glb_filename}"
    print(f"✅ Conversion done! File URL: {glb_url}")
    return jsonify({'glb_url': glb_url})

# STEP 8: File Download Endpoint
@app.route('/download/<job_id>/<subfolder>/<filename>')
def download_glb(job_id, subfolder, filename):
    directory = f"{OUTPUT_FOLDER}/{job_id}/{subfolder}"
    print(f"📤 Sending: {directory}/{filename}")
    return send_from_directory(directory, filename)

# STEP 9: Run Flask in background thread
def run_app():
    app.run(host="0.0.0.0", port=5100, use_reloader=False)

# STEP 10: Start server and keep cell alive
flask_thread = threading.Thread(target=run_app)
flask_thread.start()

# STEP 11: Keep the cell alive
try:
    while True:
        time.sleep(1)
except KeyboardInterrupt:
    print("🛑 Server stopped.")



🚀 Public API endpoint: https://841d-34-125-80-98.ngrok-free.app/convert
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5100
 * Running on http://172.28.0.12:5100
INFO:werkzeug:Press CTRL+C to quit


📸 Image saved: /content/uploads/07b5ed2f.jpg, Job ID: 07b5ed2f


INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 04:04:58] "POST /convert HTTP/1.1" 200 -


✅ Conversion done! File URL: https://841d-34-125-80-98.ngrok-free.app/download/07b5ed2f/0/mesh.glb


...

✅ Conversion done! File URL: https://841d-34-125-80-98.ngrok-free.app/download/b3b50949/0/mesh.glb


INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 04:14:36] "GET /download/b3b50949/0/mesh.glb HTTP/1.1" 200 -


📤 Sending: /content/output/b3b50949/0/mesh.glb


INFO:werkzeug:127.0.0.1 - - [09/Jun/2025 04:15:34] "GET /download/b3b50949/0/mesh.glb HTTP/1.1" 304 -


📤 Sending: /content/output/b3b50949/0/mesh.glb
🛑 Server stopped.
